In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the combined dataset
combined_data = pd.read_csv('Final_Prepared_data2.0.csv')

# Example user input
user_budget_level = 1
user_hotel_rating = 3
user_hotel_star_rating = 2
user_restaurant_rating = 3

# Filter based on user criteria
filtered_data = combined_data[
    (combined_data['budget_level'] == user_budget_level) &
    (combined_data['mmt_review_score_Hotel'] >= user_hotel_rating) &
    (combined_data['hotel_star_rating_Hotel'] >= user_hotel_star_rating) &
    (combined_data['Ratings_out_of_5_Restaurant'] >= user_restaurant_rating)
]

# Train-test split
train_data, test_data = train_test_split(filtered_data, test_size=0.2, random_state=42)


In [10]:
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances, cosine_similarity
from math import radians

# Load the combined dataset
combined_data = pd.read_csv('Final_Prepared_data2.0.csv')

# Example user input
user_budget_level = 4
user_hotel_rating = 5
user_hotel_star_rating = 5
user_restaurant_rating = 4

# Filter based on user criteria
filtered_data = combined_data[
    (combined_data['budget_level'] == user_budget_level) &
    (combined_data['mmt_review_score_Hotel'] >= user_hotel_rating) &
    (combined_data['hotel_star_rating_Hotel'] >= user_hotel_star_rating) &
    (combined_data['Ratings_out_of_5_Restaurant'] >= user_restaurant_rating)
]

# Select the first restaurant based on budget level
selected_restaurant = filtered_data.iloc[0]

# Function to calculate Haversine distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dist = haversine_distances([[lat1, lon1], [lat2, lon2]]) * 6371000  # Earth radius in meters
    return dist[1, 0]

# Apply the distance calculation to hotels
filtered_data['distance_to_restaurant'] = filtered_data.apply(
    lambda row: calculate_distance(
        selected_restaurant['Latitude_x__Restaurant'],
        selected_restaurant['Longitude_x__Restaurant'],
        row['Latitude_Hotel'],
        row['Longitude_Hotel']
    ),
    axis=1
)

# Rank hotels based on distance and additional criteria
ranked_hotels = filtered_data[filtered_data['Hotel_name'].notnull()].sort_values(
    by=['distance_to_restaurant', 'mmt_review_score_Hotel', 'hotel_star_rating_Hotel']
)

# Select the nearest hotel
nearest_hotel = ranked_hotels.iloc[0]

# Apply the distance calculation to places
filtered_data['distance_to_restaurant'] = filtered_data.apply(
    lambda row: calculate_distance(
        selected_restaurant['Latitude_x__Restaurant'],
        selected_restaurant['Longitude_x__Restaurant'],
        row['Latitude_place_0_x'],
        row['Longitude_place_0_x']
    ),
    axis=1
)

# Rank places based on distance
ranked_places = filtered_data[filtered_data['Name_Place'].notnull()].sort_values(by='distance_to_restaurant')

# Select the nearest place
nearest_place = ranked_places.iloc[0]

# Collaborative Filtering - Content Similarity
hotel_profiles = filtered_data[['mmt_review_score_Hotel', 'hotel_star_rating_Hotel', 'budget_level']]
place_profiles = filtered_data[['Rating_Place', 'Ratings_out_of_5_Restaurant', 'budget_level']]

selected_hotel_profile = selected_restaurant[['mmt_review_score_Hotel', 'hotel_star_rating_Hotel', 'budget_level']]

hotel_similarity = cosine_similarity([selected_hotel_profile], hotel_profiles)
place_similarity = cosine_similarity([selected_hotel_profile], place_profiles)

# Get top 4 hotels and places based on similarity
top_hotels = ranked_hotels.iloc[hotel_similarity.argsort()[0][::-1][:4]]
top_places = ranked_places.iloc[place_similarity.argsort()[0][::-1][:4]]

# Display the recommendations
print("Recommendations:")
print("-----------------")
print(f"Recommended Restaurant: {selected_restaurant['Restaurant_Name']}")
print(f"Nearest Hotel: {nearest_hotel['Hotel_name']}")
print(f"Nearest Place: {nearest_place['Name_Place']}")
print("\nAdditional Information:")
print("-----------------------")
print(f"Distance to Restaurant: {nearest_hotel['distance_to_restaurant']:.2f} meters")
print(f"Hotel Rating: {nearest_hotel['mmt_review_score_Hotel']}")
print(f"Hotel Star Rating: {nearest_hotel['hotel_star_rating_Hotel']}")
print(f"Restaurant Rating: {selected_restaurant['Ratings_out_of_5_Restaurant']}")

print("\nCollaborative Filtering - Top 4 Hotels:")
print(top_hotels[['Hotel_name', 'mmt_review_score_Hotel', 'hotel_star_rating_Hotel', 'distance_to_restaurant']])
print("\nCollaborative Filtering - Top 4 Places:")
print(top_places[['Name_Place', 'Ratings_out_of_5_Restaurant', 'distance_to_restaurant']])


IndexError: single positional indexer is out-of-bounds

In [8]:
from sklearn.metrics import mean_absolute_error

predicted_ratings = your_model.predict(test_data_features)  # Replace with your actual prediction method
actual_ratings = test_data['actual_ratings']

mae = mean_absolute_error(actual_ratings, predicted_ratings)


NameError: name 'your_model' is not defined